# Gitlab

## Goal

Running gitlab in a k8s cluster

## Prerequisites

* k8s cluster
* helm

## Links

* gitlab helm deployment [Quick Start Guide](https://docs.gitlab.com/charts/quickstart/index.html)
* More indepth helm deploy: [Deploy using Helm](https://docs.gitlab.com/charts/installation/deployment.html#deploy-using-helm)
* [List of Gitlab Helm charts](https://charts.gitlab.io/)
* The main [helm chart source](https://gitlab.com/gitlab-org/charts/gitlab)
* [Debugging Helm charts](https://helm.sh/docs/chart_template_guide/debugging/)
* This is a list of useful information regarding Kubernetes that the GitLab Support Team sometimes uses while troubleshooting.: [Kubernetes, GitLab, and you](https://docs.gitlab.com/ee/administration/troubleshooting/kubernetes_cheat_sheet.html#gitlab-specific-kubernetes-information)


## Adding the gitlab chart repo

In [12]:
! helm repo add gitlab https://charts.gitlab.io/
! helm repo update

"gitlab" already exists with the same configuration, skipping
Hang tight while we grab the latest from your chart repositories...
...Successfully got an update from the "gitlab" chart repository
Update Complete. ⎈Happy Helming!⎈


## dry-running

Let's see what the gitlab chart would like to deploy.

We'll use `helm install gitlab gitlab/gitlab --dry-run` and pipe the output into a file because the output is *very* long

You can add `--debug` to also get the helm values printed in your dry-run.

There is also `helm get manifest gitlab`. My understanding is that it displays the manifests *after* the deployment (*release* in helm lingo).

In [ ]:
! helm install gitlab gitlab/gitlab --dry-run --timeout 600s \
  --set global.hosts.domain=example.com \
  --set certmanager-issuer.email=me@example.com \
  > dry-run.txt

In [38]:
# Getting a rough idea of the kind and number of k8s resources by
# grepping lines with "kind: X" in them, doing some cleanup and counting them
! grep "kind: " dry-run.txt | tr -s " " | sed "s/.*kind://g" | tr -d '"' | sort | uniq -c

      1  Certificate
      1  CertificateRequest
      1  Challenge
      1  ClusterIssuer
     26  ClusterRole
     12  ClusterRoleBinding
     26  ConfigMap
      6  CustomResourceDefinition
     20  Deployment
      5  HorizontalPodAutoscaler
      4  Ingress
      1  IngressClass
      2  Issuer
      6  Job
      1  MutatingWebhookConfiguration
      1  Order
      2  PersistentVolumeClaim
      1  Pod
      8  PodDisruptionBudget
     16  Role
      8  RoleBinding
      1  Secret
     19  Service
     30  ServiceAccount
      3  StatefulSet
      1  ValidatingWebhookConfiguration


This is **a lot**

## Deploying the gitlab chart

### First attempt

I don't to use a real dmain and email adress for this lab. I tried doing a quick start with the default values:

```
helm install gitlab gitlab/gitlab \
  --set global.hosts.domain=example.com \
  --set certmanager-issuer.email=me@example.com
```

**Issues**:  

The ingress doesn't expose anything on a public IP.

I'll try to solve the issue with https://docs.gitlab.com/charts/charts/globals.html#globalingressconfigurecertmanager


### Second attempt

```
helm install gitlab gitlab/gitlab \
  --set global.hosts.domain=example.com \
  --set global.ingress.configureCertmanager=false
```

**Issues**:  

* by default, two pods with the gitlab Webservice are provisioned, but one can't be scheduled because its memory claims exceed the little test cluster
* the gitlab runner in an error status, probably because of `global.ingress.configureCertmanager=false`



### Third attempt

```
! helm install gitlab gitlab/gitlab \
  --set global.hosts.domain=example.com \
  --set global.ingress.configureCertmanager=false \
  --set gitlab.certmanager.install=false \
  --set gitlab.webservice.image.repository=registry.gitlab.com/gitlab-org/build/cng/gitlab-webservice-ce \
  --set gitlab.webservice.workhorse.image=registry.gitlab.com/gitlab-org/build/cng/gitlab-workhorse-ce \
  --set gitlab.webservice.ingress.tls.enabled=false \
  --set gitlab.deployments.default.hpa.minReplicas=1 \
  --set gitlab.deployments.default.hpa.maxReplicas=1
```


**Issues**:

* When proxying the gitlab webservice to the local machine I get this error upon trying to login/register:` 422
The change you requested was rejected`.
* `gitlab.deployments.default.hpa.maxReplicas=1` doesn't change the replicas value in the resulting hpa. Only way the change it is to edit deployed hpa: `kubectl edit hpa gitlab-webservice-default`
* `gitlab.certmanager.install=false` doesn't seem to have an effect.


**Learnings**:

* you run gitlab rake task from within the toolbox pod (`exec -it gitlab-toolbox-6646c89d6f-l226t bash`) and then `usr/local/bin/gitlab-rake --tasks`
    * [gitlab rake tasks](https://docs.gitlab.com/ee/raketasks/)

In [ ]:
! helm status gitlab | tr -s "\n"

In [44]:
# What manifests were installed ?
! helm get manifest gitlab > manifests.txt

In [ ]:
# Seems like helm installed what was promised
! grep "kind: " manifests.txt | tr -s " " | sed "s/.*kind://g" | tr -d '"' | sort | uniq -c

In [ ]:
! kubectl get ingress

In [ ]:
! kubectl describe ingress gitlab-webservice-default